In [54]:
import proj_helpers.FinObj as fin
import numpy as np
import proj_helpers.riskEngine as risk
import matplotlib.pyplot as pplot
import time

In [67]:
################################
#### Monte Carlo Parameters ####
################################
paths = 50000
pathsC = 1000000
t_del = 1

###Default setting is a two year product with two exercise dates (today and one week)
t_unit = 1 #Unit of time:  default = 1 for years, 12 for months, 360 for days etc...
time_s = 2 #Total number of time steps in t_units

#Toggle continuation estimates and payoffs to output into the regression folder
#for each timestep
#Warning! If the paths and timesteps are large these files can get VERY big.
#YOu will need this file to run the RegressionPlot notebook
outputresults = True

In [68]:
################################
###### Stock Parameters ########
################################
spot = 105.5
vol = 0.15
rate = 0.03
div = 0.0
expr = time_s/t_unit

#Share object
share1 = fin.Stock(spot,vol,t_unit,rate)

In [69]:
################################
###### Option Parameters #######
################################

strike=100
payoff=fin.birthdayVec #Define option payoff, Wedding Cake
#payoff=fin.wedgeVec #Double Digital

#Parameters for wedding cake and double digital
cake = {"rng":[1,5],"pay":[1.0,0.5]}
dd_strike = 90

#Option Parameters
myOption = fin.Option(share1,payoff,time_s,strike,t_unit,birthday=cake,wedge=dd_strike)

In [70]:
################################
#### Regression Parameters #####
################################

#Standard LSM Parameters
lsm_basis = 5

#Random Forest Parameters
rf_trees = 50
min_samples = 0.0001
max_depth = 5

In [71]:
#########################################################################
################ Get Price with Three Different Methods #################
#########################################################################
#This cell will compute the price using at least two
#different continuation value approximations

#Standard LSM
slsm = fin.priceLSM(share1,myOption,paths,time_s,t_unit,\
                    anti=True,regressor=fin.getCont,num_basis=lsm_basis,outputresults=outputresults)

print("Standard LSM Price = " + str(slsm))

#Random Forest LSM
lsmrf = fin.priceLSM(share1,myOption,paths,time_s,t_unit,\
                     anti=True,regressor=fin.getContRF,\
                     num_trees=rf_trees,max_depth=max_depth,min_samples_leaf=min_samples,outputresults=outputresults)

print("Random Forest LSM Price = " + str(lsmrf))

#Monte Carlo Tree Regression
if time_s == 2: #Monte Carlo Tree Method only valid for two time step
    mctr = fin.priceLSM(share1,myOption,paths,time_s,t_unit,\
                            anti=True,regressor=fin.getContMC,\
                            CV=fin.ControlVar(),\
                            stockC=share1,optC=myOption,pathsC=pathsC,\
                                T=time_s/t_unit,antiC=True,outputresults=outputresults)
    
    print("Monte Carlo Tree Estimation Price = " + str(mctr))


/Users/anthonymaylath/Documents/Schoolwork/NYU/Machine_Learning/CompSci/Project/src/proj_helpers/FinObj.py:244: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  betas = np.linalg.lstsq(X,Ct1)[0]


Standard LSM Price = 0.1508021814871675
Random Forest LSM Price = 0.15040719455662685
Monte Carlo Tree Estimation Price = 0.15071470477586937
